In [1]:
import pandas as pd
import numpy as np
import re
import sklearn
import xgboost as xgb
import matplotlib.pyplot as plt# Load in our libraries
import pandas as pd
import numpy as np
import re
import sklearn
import xgboost as xgb
import matplotlib.pyplot as plt
import os

from sklearn.ensemble import (RandomForestClassifier, AdaBoostClassifier, 
                              GradientBoostingClassifier, ExtraTreesClassifier)
from sklearn.svm import SVC
from sklearn.model_selection import KFold

In [2]:
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')

In [3]:
def dummies(train, test):
    columns = ['ethnicity', 'hospital_admit_source', 'icu_admit_source', 'icu_type', 'apache_3j_bodysystem', 'apache_2_bodysystem', 'cancer', 'liver_disease', 'other']
    for column in columns:
        train[column] = train[column].apply(lambda x: str(x))
        test[column] = test[column].apply(lambda x: str(x))
        good_cols = [column+'_'+i for i in train[column].unique() if i in test[column].unique()]
        train = pd.concat((train, pd.get_dummies(train[column], prefix = column)[good_cols]), axis = 1)
        test = pd.concat((test, pd.get_dummies(test[column], prefix = column)[good_cols]), axis = 1)
        del train[column]
        del test[column]
    return train, test

In [4]:
train, test = dummies(train, test)

In [5]:
train.head(2)

,Unnamed: 0,encounter_id,patient_id,hospital_id,hospital_death,age,bmi,elective_surgery,height,icu_id,...,cancer_0,cancer_1,cancer_2,cancer_3,liver_disease_0,liver_disease_2,liver_disease_1,other_1,other_0,other_2
0,1,66154,25312,118,0,68,3.123686,0,180.3,92,...,1,0,0,0,1,0,0,1,0,0
1,2,114252,59342,81,0,77,3.311273,0,160.0,90,...,1,0,0,0,1,0,0,1,0,0


In [6]:
train.shape

(91713, 112)

In [7]:
# Some useful parameters which will come in handy later on
ntrain = train.shape[0]
ntest = test.shape[0]
SEED = 0 # for reproducibility
NFOLDS = 5 # set folds for out-of-fold prediction
kf = KFold(n_splits = NFOLDS, random_state = SEED)

# Class to extend the Sklearn classifier
class SklearnHelper(object):
    def __init__(self, clf, seed=0, params=None):
        params['random_state'] = seed
        self.clf = clf(**params)

    def train(self, x_train, y_train):
        self.clf.fit(x_train, y_train)

    def predict_proba(self, x):
        c1 = self.clf.predict_proba(x)
        return c1[:,1]
    
    def fit(self,x,y):
        return self.clf.fit(x,y)
    
    def feature_importances(self,x,y):
        print(self.clf.fit(x,y).feature_importances_)
    
# Class to extend XGboost classifer



In [8]:
def get_oof(clf, x_train, y_train, x_test):
    oof_train = np.zeros((ntrain,))
    oof_test = np.zeros((ntest,))
    oof_test_skf = np.empty((NFOLDS, ntest))

    for i, (train_index, test_index) in enumerate(kf.split(x_train)):
        x_tr = x_train[train_index]
        y_tr = y_train[train_index]
        x_te = x_train[test_index]

        clf.train(x_tr, y_tr)

        oof_train[test_index] = clf.predict_proba(x_te)
        oof_test_skf[i, :] = clf.predict_proba(x_test)

    oof_test[:] = oof_test_skf.mean(axis=0)
    return oof_train.reshape(-1, 1), oof_test.reshape(-1, 1)

In [9]:
# Put in our parameters for said classifiers
# Random Forest parameters
rf_params = {
    'n_jobs': -1,
    'n_estimators': 100,
     'warm_start': True, 
     #'max_features': 0.2,
    'max_depth': 6,
    'min_samples_leaf': 2,
    'max_features' : 'auto',
    'verbose': 0
}

# Extra Trees Parameters
et_params = {
    'n_jobs': -1,
    'n_estimators':100,
    #'max_features': 0.5,
    'max_depth': 8,
    'min_samples_leaf': 2,
    'verbose': 0
}

# AdaBoost parameters
ada_params = {
    'n_estimators': 100,
    'learning_rate' : 0.1
}

# Gradient Boosting parameters
gb_params = {
    'n_estimators': 100,
     #'max_features': 0.2,
    'max_depth': 5,
    'min_samples_leaf': 2,
    'verbose': 0
}

# Support Vector Classifier parameters 
#svc_params = {
#    'kernel' : 'sigmoid',
#    'C' : 0.025,
#    'probability' : True
#    }

In [10]:
# Create Numpy arrays of train, test and target ( Survived) dataframes to feed into our models
y_train = train['hospital_death'].ravel()
train1 = train.drop(['hospital_death'], axis=1)
test1 = test.drop(['hospital_death'], axis=1)
x_train = train1.values # Creates an array of the train data
x_test = test1.values # Creats an array of the test data

In [11]:
rf = SklearnHelper(clf=RandomForestClassifier, seed=SEED, params=rf_params)
et = SklearnHelper(clf=ExtraTreesClassifier, seed=SEED, params=et_params)
ada = SklearnHelper(clf=AdaBoostClassifier, seed=SEED, params=ada_params)
gb = SklearnHelper(clf=GradientBoostingClassifier, seed=SEED, params=gb_params)
#svc = SklearnHelper(clf=SVC, seed=SEED, params=svc_params)

In [12]:
# Create our OOF train and test predictions. These base results will be used as new features
et_oof_train, et_oof_test = get_oof(et, x_train, y_train, x_test) # Extra Trees
print("testin 2")
rf_oof_train, rf_oof_test = get_oof(rf,x_train, y_train, x_test) # Random Forest
print("testing 3")
ada_oof_train, ada_oof_test = get_oof(ada, x_train, y_train, x_test) # AdaBoost 
print("testing 4")
gb_oof_train, gb_oof_test = get_oof(gb,x_train, y_train, x_test) # Gradient Boost
#print("testing 5")
#svc_oof_train, svc_oof_test = get_oof(svc,x_train, y_train, x_test) # Support Vector Classifier

print("Training is complete")

testin 2


C:\Users\anany\Anaconda3\envs\envTF113\lib\site-packages\sklearn\ensemble\forest.py:307: UserWarning: Warm-start fitting without increasing n_estimators does not fit new trees.
  warn("Warm-start fitting without increasing n_estimators does not "
C:\Users\anany\Anaconda3\envs\envTF113\lib\site-packages\sklearn\ensemble\forest.py:307: UserWarning: Warm-start fitting without increasing n_estimators does not fit new trees.
  warn("Warm-start fitting without increasing n_estimators does not "
C:\Users\anany\Anaconda3\envs\envTF113\lib\site-packages\sklearn\ensemble\forest.py:307: UserWarning: Warm-start fitting without increasing n_estimators does not fit new trees.
  warn("Warm-start fitting without increasing n_estimators does not "
C:\Users\anany\Anaconda3\envs\envTF113\lib\site-packages\sklearn\ensemble\forest.py:307: UserWarning: Warm-start fitting without increasing n_estimators does not fit new trees.
  warn("Warm-start fitting without increasing n_estimators does not "


testing 3
testing 4
Training is complete


In [13]:
rf_feature = rf.feature_importances(x_train,y_train)
et_feature = et.feature_importances(x_train, y_train)
ada_feature = ada.feature_importances(x_train, y_train)
gb_feature = gb.feature_importances(x_train,y_train)

[1.77183251e-03 9.46319152e-04 8.30229972e-04 1.18869890e-03
 9.90810102e-03 1.85192531e-03 4.04015717e-03 7.78356259e-04
 1.54463958e-03 2.41151372e-03 1.46185977e-02 3.72661006e-02
 5.02986611e-03 4.19374369e-05 6.20292953e-03 4.71325260e-03
 7.99324309e-03 1.30860069e-02 3.52475978e-02 4.71278252e-03
 2.12874427e-03 3.28625903e-02 5.29346278e-02 3.30465846e-02
 3.45682478e-02 1.40859696e-02 1.92880903e-03 5.58640111e-03
 2.76900736e-03 1.62115021e-03 2.69738030e-03 2.25708282e-03
 5.39352727e-03 5.22810252e-03 2.81830754e-03 4.56882148e-03
 1.10893914e-03 9.32614457e-03 4.86656259e-03 9.02956854e-03
 1.87272072e-02 3.37893912e-02 6.97886780e-02 1.15180191e-02
 1.40323068e-02 1.83804762e-01 2.04157524e-01 0.00000000e+00
 5.77144241e-02 1.70672469e-04 1.44401432e-05 8.76061166e-06
 1.02416322e-04 4.07438673e-05 0.00000000e+00 8.45253006e-05
 1.93012977e-04 1.78827958e-04 9.88092544e-04 2.67529193e-04
 5.62143977e-05 0.00000000e+00 4.29014734e-05 0.00000000e+00
 0.00000000e+00 2.870619

C:\Users\anany\Anaconda3\envs\envTF113\lib\site-packages\sklearn\ensemble\forest.py:307: UserWarning: Warm-start fitting without increasing n_estimators does not fit new trees.
  warn("Warm-start fitting without increasing n_estimators does not "


[8.13786865e-04 3.99349491e-04 5.87860047e-04 7.77109109e-04
 1.01004192e-02 8.40343786e-04 1.81126758e-02 5.49303602e-04
 8.66198780e-04 8.47210016e-04 9.79386959e-03 1.34201029e-02
 1.27597443e-02 4.41705901e-04 5.79951483e-03 3.47121598e-03
 8.55654776e-03 4.15755177e-02 8.23754295e-02 5.43604268e-03
 1.17221794e-03 1.82631995e-02 4.72212348e-02 1.64314605e-02
 4.01025607e-02 1.36959696e-02 2.83129533e-03 5.25860114e-03
 2.56533915e-03 1.10382831e-03 2.13277013e-03 1.40602368e-03
 5.42404134e-03 5.61375051e-03 4.22971265e-03 5.66781819e-03
 9.42024428e-04 8.55339764e-03 6.26873135e-03 6.08683884e-03
 1.02954289e-02 2.99344047e-02 4.65083261e-02 6.06985969e-03
 5.95690572e-03 1.32583931e-01 1.43344793e-01 1.85346912e-06
 1.03342710e-01 5.72651907e-04 1.01403394e-04 2.97446170e-04
 4.91845362e-04 1.68524924e-04 1.43985707e-04 4.07352483e-04
 1.70953473e-03 1.27046841e-03 4.00134548e-03 9.18223608e-04
 3.07664315e-04 2.36916228e-04 1.09030380e-04 3.26854991e-06
 1.21838012e-04 1.982977

In [14]:
base_predictions_train = pd.DataFrame( {'RandomForest': rf_oof_train.ravel(),
     'ExtraTrees': et_oof_train.ravel(),
     'AdaBoost': ada_oof_train.ravel(),
      'GradientBoost': gb_oof_train.ravel()
    })
base_predictions_train.head()

,RandomForest,ExtraTrees,AdaBoost,GradientBoost
0,0.078155,0.092912,0.445817,0.082193
1,0.410651,0.319828,0.507072,0.780385
2,0.024704,0.024721,0.384096,0.007017
3,0.056990,0.045552,0.429606,0.035800
4,0.027839,0.030305,0.406569,0.012461


In [15]:
x_train = np.concatenate(( et_oof_train, rf_oof_train, ada_oof_train, gb_oof_train), axis=1)
x_test = np.concatenate(( et_oof_test, rf_oof_test, ada_oof_test, gb_oof_test), axis=1)

In [16]:
gbm = xgb.XGBClassifier(
    #learning_rate = 0.02,
 n_estimators= 100,
 max_depth= 4,
 min_child_weight= 2,
 #gamma=1,
 gamma=0.0,                        
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread= -1,
 scale_pos_weight=1).fit(x_train, y_train)
test['hospital_death'] = 1 - gbm.predict_proba(x_test)

In [18]:
tackingSubmission = test[["encounter_id","hospital_death"]].to_csv("submission_stacking.csv",index=False)